In [9]:
import numpy as np
import tifffile as tiff
import gc
def untile(im):
    nim=[]
    n=5
    for i in range(n):
        for j in range(n):
            nim.append(im[:,:,(j)*1024:(j+1)*1024,(i)*1024:(i+1)*1024])
    return nim

def selmax(im):
    nim=np.zeros((25,4,1024,1024))
    num=0
    for i in im:
        nim[num]=i[0]
        for j in i:
            if np.mean(j[0])>np.mean(nim[num][0]):
                nim[num]=j
        num+=1
    return np.transpose(nim,(0,2,3,1))
path = "../Documents/Microscopie/2022-04-21_BioSamples_Acquisition/"
dataim=[tiff.imread(path + "01_NDD+interne_5-5_1024-1024_5s.lsm"),
        tiff.imread(path + "02_NDD+interne_5-5_1024-1024_5s.lsm"),
        tiff.imread(path + "03_NDD+interne_5-5_1024-1024_5s.lsm"),
        tiff.imread(path + "04_NDD+interne_5-5_1024-1024_5s.lsm"),
        tiff.imread(path + "05_NDD+interne_5-5_1024-1024_5s.lsm"),
        tiff.imread(path + "06_NDD+interne_5-5_1024-1024_5s.lsm"),
        tiff.imread(path + "07_NDD+interne_5-5_1024-1024_5s.lsm"),
       ]
imgs=None
for im in dataim:
    im2=untile(im)
    im3=selmax(im2).astype("float32")
    if imgs is None:
        imgs=im3
    else:imgs=np.concatenate((imgs,im3),axis=0)
    del im2,im3
del dataim
gc.collect()

907

In [13]:
print(np.shape(imgs))

(4, 1024, 1024, 4)


In [6]:
fres=np.zeros([7,5120,5120,4])
for k in range(7):
    for j in range (5):
        for i in range (5):
            fres[k,i*1024:i*1024+1024,j*1024:j*1024+1024]=(imgs[25*k+5*j+i])

In [12]:
print(np.shape(fres))

(7, 5120, 5120, 4)


In [14]:
for i in range(len(fres)):
    tiff.imsave(path+"0%d_NDD+interne_5-5_1024-1024_1s.tif"%(i+1),fres[i].astype("uint8"))